# Open Street Map

Sources:
- [API Documentation](https://wiki.openstreetmap.org/wiki/API_v0.6)

In [3]:
!pip install -q osmnx

In [2]:
import requests
import osmnx as ox
import geopandas as gpd
from tqdm import tqdm
from shapely.geometry import Point

In [3]:
tqdm.pandas()

## Load data

In [5]:
gdf = gpd.read_file("data/administrative_boundaries/gadm/gadm41_SDN_2.json")

## Extract lat-long from multipolygon format

In [10]:
# Ensure geometries are in EPSG:4326 (lat/lon)
gdf = gdf.to_crs(epsg=4326)

In [12]:
def convert_coordinates_format(geometry):
    """
    Convert coordinates from GeoDataFrame geometry (tuple format) to the required format.
    
    Parameters:
        geometry (shapely.geometry): The geometry object (Polygon or MultiPolygon).
        
    Returns:
        list: Coordinates in the required format, i.e., a list of lists of [longitude, latitude].
    """
    if geometry.geom_type == 'MultiPolygon':
        # convert each Polygon within the MultiPolygon to the required format
        return [
            [list(polygon.exterior.coords) for polygon in geometry.geoms]
        ]
    elif geometry.geom_type == 'Polygon':
        # convert the Polygon to the required format
        return [list(geometry.exterior.coords)]
    else:
        return []

In [14]:
gdf["coordinates"] = gdf["geometry"].progress_apply(lambda x: convert_coordinates_format(x))

100%|██████████| 80/80 [00:00<00:00, 3541.10it/s]


In [16]:
def get_centroid_coordinates(gdf):
    """
    Get the centroid of each region in the GeoDataFrame.
    
    Args:
    - gdf (GeoDataFrame): A GeoDataFrame with polygon or multipolygon geometries.
    
    Returns:
    - list: A list of (latitude, longitude) tuples for each region's centroid.
    """
    centroids = []
    
    for _, row in gdf.iterrows():
        geometry = row['geometry']
        
        if geometry.is_valid:
            centroid = geometry.centroid
            centroids.append((centroid.y, centroid.x))  # (lat, lon)
    
    return centroids

In [18]:
gdf["centroid_coordinates"] = get_centroid_coordinates(gdf)

## Extract OSM data

In [21]:
def get_bbox_from_multipolygon(multipolygon):
    """
    Calculate the bounding box (BBOX) from a GeoPandas multipolygon geometry.
    
    Parameters:
        multipolygon (shapely.geometry.multipolygon.MultiPolygon): The multipolygon geometry.
    
    Returns:
        list: Bounding box in the format [min_lat, min_lon, max_lat, max_lon].
    """
    bounds = multipolygon.bounds  # [min_lon, min_lat, max_lon, max_lat]
    return [bounds[1], bounds[0], bounds[3], bounds[2]]  # Reorder to [min_lat, min_lon, max_lat, max_lon]

In [23]:
def fetch_node_coordinates(node_ids, chunk_size=50):
    """
    Fetch coordinates for a list of node IDs from OpenStreetMap using the Overpass API.
    
    Parameters:
        node_ids (list): List of node IDs to retrieve coordinates for.
        chunk_size (int): The number of node IDs to query in each request. Default is 50.
        
    Returns:
        list: A list of coordinates (lat, lon) for each node.
    """
    overpass_url = "https://overpass-api.de/api/interpreter"
    coordinates = []
    
    # Split the node IDs into chunks of size chunk_size
    for i in range(0, len(node_ids), chunk_size):
        chunk = node_ids[i:i + chunk_size]
        
        # Create Overpass query for each chunk of nodes
        query = f"""
        [out:json];
        node({','.join(map(str, chunk))});
        out body;
        """
        print(f"Querying for nodes: {chunk}")
        print(f"Query: {query}")
        
        response = requests.get(overpass_url, params={'data': query})
        
        if response.status_code == 200:
            # Extract node data and store coordinates
            node_data = response.json()
            nodes = {node['id']: (node['lat'], node['lon']) for node in node_data['elements']}
            
            # Append the coordinates of nodes that were found
            coordinates.extend([nodes[node_id] for node_id in chunk if node_id in nodes])
        else:
            print(f"Failed to fetch data for nodes {chunk}. Status code: {response.status_code}")
    
    return coordinates

In [25]:
def get_osm_data_from_bbox(bbox):
    """
    Fetch infrastructure and roads data from OSM using Overpass API for a given BBOX.
    
    Parameters:
        bbox (list): Bounding box in the format [min_lat, min_lon, max_lat, max_lon].
    
    Returns:
        dict: JSON response from the Overpass API containing OSM data.
    """
    overpass_url = "https://overpass-api.de/api/interpreter"
    # Overpass QL query for roads and infrastructure within the BBOX
    query = f"""
    [out:json];
    (
      way["highway"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["railway"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
      way["waterway"]({bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]});
    );
    out body;
    >;
    out skel qt;
    """
    response = requests.get(overpass_url, params={'data': query})
    if response.status_code == 200:
        osm_data = response.json()
        return osm_data.get('elements', [])
    else:
        response.raise_for_status()

In [27]:
osm_data = list()

In [29]:
for _, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
    multipolygon = row['geometry']
    bbox = get_bbox_from_multipolygon(multipolygon)
    osm_data.append(get_osm_data_from_bbox(bbox)) 

100%|██████████| 80/80 [02:15<00:00,  1.69s/it]


In [30]:
gdf["osm_data"] = osm_data

## Saving the data

In [32]:
gdf.drop(columns='geometry').to_csv('data/openstreetmap/openstreetmap_sdn_by_state_district.csv', index=False)

In [33]:
gdf.drop(columns='geometry').to_json('data/openstreetmap/openstreetmap_sdn_by_state_district.json', orient='records')

In [34]:
gdf.to_file("data/openstreetmap/openstreetmap_sdn_by_state_district.geojson", driver="GeoJSON")